# 基础理论

### Apache RocketMQ is a distributed messaging and streaming data platform.

* Architecture: It consists of four parts: name servers, brokers, producers and consumers. Each of them can be horizontally extended without a single Point of Failure. 
<img src="../../images/javaee/rmq-basic-arc.png" width="600px">
    * <b>NameServer Cluster</b>: Name Servers provide lightweight service discovery and routing. Each Name Server records full routing information, provides corresponding reading and writing service, and supports fast storage expansion.
        * Broker Management, NameServer accepts the register from Broker cluster and provides heartbeat mechanism to check whether a broker is alive.
        * Routing Management, each NameServer will hold whole routing info about the broker cluster and the queue info for clients query.
    * <b>Broker Cluster</b>: Brokers take care of message storage by providing lightweight TOPIC and QUEUE mechanisms. They support the Push and Pull model, contains fault tolerance mechanism (2 copies or 3 copies), and provides strong padding of peaks and capacity of accumulating hundreds of billion messages in their original time order. In addition, Brokers provide disaster recovery, rich metrics statistics, and alert mechanisms, all of which are lacking in traditional messaging systems.
        * Remoting Module, the entry of broker, handles the requests from clients.
        * Client Manager, manages the clients (Producer/Consumer) and maintains topic subscription of consumer.
        * Store Service, provides simple APIs to store or query message in physical disk.
        * HA Service, provides data sync feature between master broker and slave broker.
        * Index Service, builds index for messages by specified key and provides quick message query.
    * <b>Producer Cluster</b>: Producers support distributed deployment. Distributed Producers send messages to the Broker cluster through multiple load balancing modes. The sending processes support fast failure and have low latency.
    * <b>Consumer Cluster</b>: Consumers support distributed deployment in the Push and Pull model as well. It also supports cluster consumption and message broadcasting. It provides real-time message subscription mechanism and can meet most consumer requirements.





****

# 应用实战
* 详细应用示例: https://jaskey.github.io/
* 使用示例: https://rocketmq.apache.org/docs/quick-start/
* 部署示例: https://rocketmq.apache.org/docs/rmq-deployment/
* 命令行工具集: https://rocketmq.apache.org/docs/cli-admin-tool/

****

# 设计实现原理: comparing to Kafka for understanding of these concept
* Motivation: https://rocketmq.apache.org/docs/motivation/
* Design modal:
<img src="../../images/javaee/rmq-model.png" width="500px">
* <b>Producer</b>: A producer sends messages generated by the business application systems to brokers.
    * Producer Group: 
        * Producers of the same role are grouped together. A different producer instance of the same producer group may be contacted by a broker to commit or roll back a transaction in case the original producer crashed after the transaction.
        * Considering the provided producer is sufficiently powerful at sending messages, only one instance is allowed per producer group to avoid unnecessary initialization of producer instances.
* <b>Consumer</b>: A Consumer pulls messages from brokers and feeds them into application.
    * PullConsumer: Pull consumer actively pulls messages from brokers. Once batches of messages are pulled, user application initiates consuming process.
    * PushConsumer: Push consumer encapsulates message pulling, consuming progress and maintaining other work inside, leaving a callback interface to end user to implement which will be executed on message arrival.
    * Consumer Group:
        * consumers of the exactly same role are grouped together and named Consumer Group. Consumer Group is a great concept with which achieving goals of load-balance and fault-tolerance, in terms of message consuming, is super easy.
        * consumer instances of a consumer group must have exactly the same topic subscription
    * <b>Consumer offset</b>: https://jaskey.github.io/blog/2017/01/25/rocketmq-consume-offset-management/
        * <i>`Kafka处理是通过消费端消费完消息后主动提交consumer offset到broker完成持久化`</i>
        * <i>`RocketMQ处理是通过消费端消费完消息后通过定时器提交consumer offset到broker完成持久化`</i>
* <b>Topic</b>: Topic is a category in which producers deliver messages and consumers pull messages.
    * a topic may have zero, one or multiple producers that sends messages to it; conversely, a producer can send messages of different topics.
    * a topic may be subscribed by zero, one or multiple consumer groups. And a consumer group, similarly, may subscribe to one or more topics as long as instances of this group keep their subscription consistent.
* <b>Message</b>: Message is the information to be delivered. A message must have a topic, which can be interpreted as address of your letter to mail to. A message may also have an optional tag and extra key-value pairs.
* <b>Message Queue</b>: Topic is partitioned into one or more sub-topics, “message queues”.
* <b>Tag</b>: Tag, in other words sub-topic, provides extra flexibility to users. With tag, messages with different purposes from the same business module may have the same topic and different tag. Tags would be helpful to keep your code clean and coherent, and tags also can facilitate the query system RocketMQ provides.
* <b>Broker</b>: Broker is a major component of the RocketMQ system. It receives messages sent from producers, store them and prepare to handle pull requests from consumers. It also stores message related meta data, including consumer groups, consuming progress offsets and topic / queue info.
* <b>Name Server</b>: Name server serves as the routing information provider. Producer/Consumer clients look up topics to find the corresponding broker list.
* <b>Message Order</b>
    * Orderly: 
        * Consuming messages orderly means messages are consumed the same order they are sent by producers for each message queue. If you are dealing with scenario that global order is mandatory, make sure the topic you use has only one message queue.
        * If consuming orderly is specified, the maximum concurrency of message consuming is the number of message queues subscribed by the consumer group.
    * Concurrently
        * When consuming messages concurrently, maximum concurrency of message consuming is only limited by thread pool specified for each consumer client.
        * Message order is no longer guaranteed in this mode.